In [1]:
from __future__ import print_function, division, absolute_import
import os
import sys
sys.path.insert(0, "C:\\Users\\Matt\\Desktop\\orbithunter\\")
from orbithunter import *
import numpy as np
# from scipy.fft import irfft,rfft,rfftn,irfftn,fftn,ifftn,fft,ifft,fft2,rfft2
import itertools
from math import pi
# from orbithunter.glue import *
import matplotlib.pyplot as plt
from scipy.fft import rfft, irfft
from functools import lru_cache
from orbithunter.arrayops import swap_modes
%matplotlib inline

import time

In [2]:
orbit = read_h5("C:\\Users\\Matt\\Desktop\\orbithunter_test_data\\none_000_021.h5", data_format='kstori', state_type='field')

In [27]:
int_orbit = orbit.__class__(state=orbit.state[-1,:].reshape(1,-1), state_type=orbit.state_type,
                            T=orbit.T, L=orbit.L, S=orbit.S).convert(to='s_modes')

In [5]:
def dx_spatial_modes(self, power=1):
    modes = self.convert(to='s_modes').state
    # Elementwise multiplication of modes with frequencies, this is the derivative.
    dxn_modes = np.multiply(self.elementwise_dxn(self.parameters, power=power), modes)

    # If the order of the differentiation is odd, need to swap imaginary and real components.
    if np.mod(power, 2):
        dxn_modes = swap_modes(dxn_modes, dimension='space')

    return self.__class__(state=dxn_modes, state_type='s_modes', T=self.T, L=self.L)

In [32]:
(int_orbit.wave_vector(int_orbit.parameters, power=2) - int_orbit.wave_vector(int_orbit.parameters, power=4)).reshape(-1,1)

array([[ 0.04840059],
       [ 0.16238818],
       [ 0.2483203 ],
       [ 0.15012615],
       [-0.35069341],
       [-1.53506582],
       [-3.74634685]])

array([[ 0.04840059,  0.16238818,  0.2483203 ,  0.15012615, -0.35069341,
        -1.53506582, -3.74634685,  0.04840059,  0.16238818,  0.2483203 ,
         0.15012615, -0.35069341, -1.53506582, -3.74634685]])

In [29]:
-1*int_orbit.jac_lin().diagonal().reshape(-1,1)

array([[ 0.04840059],
       [ 0.16238818],
       [ 0.2483203 ],
       [ 0.15012615],
       [-0.35069341],
       [-1.53506582],
       [-3.74634685],
       [ 0.04840059],
       [ 0.16238818],
       [ 0.2483203 ],
       [ 0.15012615],
       [-0.35069341],
       [-1.53506582],
       [-3.74634685]])

# original beneath this

In [ ]:
tmax = T
nmax = int(tmax/h)
g = -0.5j*k
uu = np.zeros([M,nmax],float)
tt = np.zeros([nmax,1],float)
vv = np.zeros([M,nmax],complex)
uu[0:M,0] = u[0:M,0]
vv[0:M,0]= v[0:M,0]

for step in range(0,nmax):
    t = step*h
    Nv = g*fft(np.real(ifft(v, axis=0)*np.sqrt(M))**2,axis=0) / np.sqrt(M)
    a = E2*v + Q*Nv
    Na = g*fft(np.real(ifft(a, axis=0)*np.sqrt(M))**2,axis=0) / np.sqrt(M)
    b = E2*v + Q*Na
    Nb = g*fft(np.real(ifft(b, axis=0)*np.sqrt(M))**2,axis=0) / np.sqrt(M)
    c = E2*a + Q*(2.0*Nb-Nv)
    Nc = g*fft(np.real(ifft(c, axis=0)*np.sqrt(M))**2,axis=0) / np.sqrt(M)
    v = E*v + Nv*f1 + 2.0*(Na+Nb)*f2 + Nc*f3
    v[0] = 0
    v[int(M // 2)] = 0
    u = np.sqrt(M)*np.real(ifft(v,axis=0))
    uu[0:M,step] = u[0:M,0]
    tt[step] = t
    if np.mod(step,3000)==0:
        print(t)
uu = np.flipud(uu.transpose())
# vv = np.flipud(vv.transpose())
# plt.figure()
# plt.imshow(uu,cmap='jet')
# plt.show()
orbit_tmp = (uu,nstp,M,T,L,S)
orbit = orbit_init.relevant_symmetry_operation(orbit_tmp,symmetry=symmetry)